In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns           
import matplotlib
import matplotlib.pyplot as plt  
import os
from sklearn import preprocessing 
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

%matplotlib inline     
sns.set(color_codes=True)

In [ ]:
carsons_path = 'C:\\Users\\Carson\\Desktop\\cmpe351'
# carsons_path = "C:\\Users\\thoma\\Desktop\\Data Airbnb"

listings = pd.read_csv(os.path.join(carsons_path, "listings.csv"))
sum_listings = pd.read_csv(os.path.join(carsons_path, "listings summary.csv"))
sum_nbhd = pd.read_csv(os.path.join(carsons_path, "neighbourhoods summary.csv"))
sum_rev = pd.read_csv(os.path.join(carsons_path, "reviews summary.csv"))
revs = pd.read_csv(os.path.join(carsons_path, "reviews.csv"))


minimum_nights              minimum number of night stay for the listing (calendar rules may be different)
minimum_minimum_nights      the smallest minimum_night value from the calender (looking 365 nights in the future)
maximum_minimum_nights      the largest minimum_night value from the calender (looking 365 nights in the future)
minimum_nights_avg_ntm      the average minimum_night value from the calender (looking 365 nights in the future)

### RQ2: How does the minimum number of nights that customers must book affect the likelihood of a property being vacant?

In [ ]:
sum_nbhd.columns

In [ ]:
sum_listings

In [ ]:
sum_rev

In [ ]:
revs

In [ ]:
print(listings.columns)

In [ ]:
listings['host_is_superhost'].head(5)

In [ ]:
sum_nbhd.head()

In [ ]:
sum_rev.head()

In [ ]:
revs.head()

In [ ]:
listings['minimum_minimum_nights']

In [ ]:
listings['minimum_nights']

In [ ]:
(listings['minimum_minimum_nights']-listings['minimum_nights']).describe()

In [ ]:
columns = ['minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365']

In [ ]:
pd.DataFrame.hist(data = sum_listings[sum_listings['minimum_nights']<21], column='minimum_nights', bins=10)

In [ ]:
pd.DataFrame.hist(data = listings[listings['calculated_host_listings_count']>1], column='calculated_host_listings_count', bins=10)

In [ ]:
sum_listings[sum_listings['minimum_nights']>21].count()

In [ ]:
dud_listings =listings[listings['minimum_nights']<21]

min_nights = dud_listings['minimum_nights']
avail_356 = dud_listings['availability_365']
avail_90 = dud_listings['availability_90']
avail_60 = dud_listings['availability_60']
avail_30 = dud_listings['availability_30']

corr_365 = avail_356.corr(min_nights)
corr_90 = avail_90.corr(min_nights)
corr_60 = avail_60.corr(min_nights)
corr_30 = avail_30.corr(min_nights)

print(corr_365)
print(corr_90)
print(corr_60)
print(corr_30)

In [ ]:
listings = listings[listings['host_is_superhost'] == 't']
len(listings)

In [ ]:
dud_listings = listings[listings['minimum_nights'] > 100]

min_nights = dud_listings['minimum_nights']
avail_356 = dud_listings['availability_365']
avail_90 = dud_listings['availability_90']
avail_60 = dud_listings['availability_60']
avail_30 = dud_listings['availability_30']

corr_365 = avail_356.corr(min_nights)
corr_90 = avail_90.corr(min_nights)
corr_60 = avail_60.corr(min_nights)
corr_30 = avail_30.corr(min_nights)

print(corr_365)
print(corr_90)
print(corr_60)
print(corr_30)

does not appear to be a substantial correlation between number of minimum nights and availability


In [ ]:
unbooked = listings[listings['availability_365'] > 3]
unbooked

In [ ]:
one_night = listings[listings['minimum_nights'] == 1]
print(len(one_night))
one_night = one_night[one_night['availability_365']>3]
one_night

In [ ]:
pd.DataFrame.hist(data = listings[listings['minimum_nights']<7], column='availability_365', bins=100)


In [ ]:
pd.DataFrame.hist(data = listings[listings['minimum_nights']>7], column='availability_365', bins=100)

In [ ]:
more_than_a_week = listings[listings['minimum_nights']>7]
abs(len(more_than_a_week[more_than_a_week['availability_365']>3]) - len(more_than_a_week))/len(more_than_a_week)

In [ ]:
less_than_a_week = listings[listings['minimum_nights']<7]
abs(len(less_than_a_week[less_than_a_week['availability_365']<30]) - len(less_than_a_week))/len(less_than_a_week)

## Find correlations between vacancy and factors while only looking at datapoints that exist in the middle of the set. i.e exclude all and no vacancy

In [ ]:
df_small = listings[listings['availability_365'] > 60 ]
df_small = df_small[df_small['availability_365'] < 305 ]
pd.DataFrame.hist(data = df_small, column='availability_365', bins=100)


In [ ]:
numeric_cols = df_small.select_dtypes(include=['number'])

df_small = df_small.drop(columns=df_small.select_dtypes(exclude=['number']).columns)

In [ ]:
cols = ['id', 'scrape_id', 'description', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

for col in cols:
    if abs(df_small['availability_365'].corr(df_small[col])) > 0.1:
        print('column: ', col)
        print('correlation: ', df_small['availability_365'].corr(df_small[col]))

these are not promising results, attempt to try classification model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

# listings
non_numeric_cols = listings.select_dtypes(exclude=['number'])
data = listings.drop(columns=listings.select_dtypes(include=['number']).columns)
cols_to_drop = ['last_scraped','amenities','listing_url', 'source', 'name','neighborhood_overview', 'picture_url', 'host_url', 'host_name', 'host_since', 'host_about','host_thumbnail_url', 'host_picture_url','bathrooms_text','calendar_last_scraped', 'first_review', 'last_review']
data = data.drop(columns=cols_to_drop)

data = pd.get_dummies(data)

# Split the data into features (X) and target (y)
# X = data.drop('Target_Yes', axis=1)  # Features
# y = df['Target_Yes']  # Target variable

In [ ]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, listings['availability_365'], test_size=0.2, random_state=42)

# Create a logistic regression model
clf = DecisionTreeClassifier()

# Train the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


a preliminary attempt to use categorical variables does not yield promising results either, try again with smaller df

In [ ]:
df_small = listings[listings['availability_365'] > 60 ]
df_small = df_small[df_small['availability_365'] < 305 ]
non_numeric_cols = df_small.select_dtypes(exclude=['number'])
data = df_small.drop(columns=df_small.select_dtypes(include=['number']).columns)
print(data.columns)
cols_to_drop = ['last_scraped','amenities','listing_url', 'source', 'name','neighborhood_overview', 'picture_url', 'host_url', 'host_name', 'host_since', 'host_about','host_thumbnail_url', 'host_picture_url','bathrooms_text','calendar_last_scraped', 'first_review', 'last_review']
data = data.drop(columns=cols_to_drop)

data = pd.get_dummies(data)

# Split the data into features (X) and target (y)
# X = data.drop('Target_Yes', axis=1)  # Features
# y = df['Target_Yes']  # Target variable

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, df_small['availability_365'], test_size=0.2, random_state=42)

# Create a logistic regression model
clf = DecisionTreeClassifier()

# Train the model on the training data
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


did wayyyy worse, that is also bad

In [ ]:
pd.DataFrame.hist(data = listings, column='availability_365', bins=100)

## Observe there is some recurrence here, at pseudo approximate intervals there is some normality, attempt to extract this

In [ ]:
df_50_to_100 = listings[listings['availability_365'] > 50 ]
df_50_to_100 = df_50_to_100[df_50_to_100['availability_365'] < 100 ]
pd.DataFrame.hist(data = df_50_to_100, column='availability_365', bins=20)

In [ ]:
df_125_to_200 = listings[listings['availability_365'] > 125 ]
df_125_to_200 = df_125_to_200[df_125_to_200['availability_365'] < 200 ]
pd.DataFrame.hist(data = df_125_to_200, column='availability_365', bins=20)

In [ ]:
df_225_to_300 = listings[listings['availability_365'] > 225 ]
df_225_to_300 = df_225_to_300[df_225_to_300['availability_365'] < 300 ]
pd.DataFrame.hist(data = df_225_to_300, column='availability_365', bins=20)

not quite as normal as expected, will still look into correlations

In [ ]:
cols = ['id', 'scrape_id', 'description', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']


for col in cols:
    if abs(df_50_to_100['availability_365'].corr(df_50_to_100[col])) > 0.1:
        print('column: ', col)
        print('correlation: ', df_50_to_100['availability_365'].corr(df_50_to_100[col]))

In [ ]:
cols = ['id', 'scrape_id', 'description', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

for col in cols:
    if abs(df_125_to_200['availability_365'].corr(df_125_to_200[col])) > 0.1:
        print('column: ', col)
        print('correlation: ', df_125_to_200['availability_365'].corr(df_125_to_200[col]))

In [ ]:
cols = ['id', 'scrape_id', 'description', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_updated', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

tempCols = ['id', 'scrape_id', 'description', 'host_id', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'calendar_updated', 'number_of_reviews', 'number_of_reviews_ltm', 'number_of_reviews_l30d',
       'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']


for col in cols:
    if abs(df_225_to_300['availability_365'].corr(df_225_to_300[col])) > 0.1:
        print('column: ', col)
        print('correlation: ', df_225_to_300['availability_365'].corr(df_225_to_300[col]))

no bueno, lets try to extract some date features

In [ ]:
listings.columns

In [ ]:
listings.head()

In [ ]:
listings['last_review'] = pd.to_datetime(listings['last_review'])
listings['calendar_last_scraped'] = pd.to_datetime(listings['calendar_last_scraped'])
listings['calendar_updated'] = pd.to_datetime(listings['calendar_updated'])
listings['host_since'] = pd.to_datetime(listings['host_since'])
listings['last_scraped'] = pd.to_datetime(listings['last_scraped'])
listings['first_review'] = pd.to_datetime(listings['first_review'])

In [ ]:
plt.figure(figsize=(8, 12))
heatmap = sns.heatmap(listings.drop(columns=tempCols).corr()[['availability_30', 'availability_60', 'availability_90', 'availability_365']], vmin=-1, vmax=1, annot=True, cmap='BrBG')
heatmap.set_title('Features Correlating with Availability', fontdict={'fontsize':18}, pad=16);

In [ ]:
cols = ['id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'maximum_nights','minimum_maximum_nights',
       'maximum_maximum_nights',
       'maximum_nights_avg_ntm', 'calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews',
       'number_of_reviews_ltm', 'number_of_reviews_l30d', 'first_review',
       'last_review', 'review_scores_rating', 'review_scores_accuracy',
       'review_scores_cleanliness', 'review_scores_checkin',
       'review_scores_communication', 'review_scores_location',
       'review_scores_value', 'license', 'instant_bookable',
    #    'calculated_host_listings_count',
       'calculated_host_listings_count_entire_homes',
       'calculated_host_listings_count_private_rooms',
       'calculated_host_listings_count_shared_rooms', 'reviews_per_month']

In [ ]:
dfs = [listings, df_50_to_100, df_125_to_200, df_225_to_300]
targets = ['availability_30', 'availability_60', 'availability_90', 'availability_365']
names = ['listings', 'df_50_to_100', 'df_125_to_200', 'df_225_to_300']
for i in range(len(dfs)):
    for target in targets:
        X_train, X_test, y_train, y_test = train_test_split(dfs[i].drop(columns = cols), dfs[i][target], test_size=0.2, random_state=20166925)
        gb_regressor = GradientBoostingRegressor(n_estimators=400, max_depth=8, learning_rate=0.1, random_state=42)
        gb_regressor.fit(X_train, y_train)

            # Predict on the test set
        predictions = gb_regressor.predict(X_test)

            # Evaluate the model
            # mse = mean_squared_error(y_test, predictions)
            # print("Mean Squared Error:", mse)

            # print(y_test)
            # print(len(predictions))
            # print(len(df_test))
        mse = mean_squared_error(y_test, predictions)**0.5
        print('df: ', names[i])
        print('target: ', target)
        print('MSE: ', mse)

In [ ]:
dfs = [listings, df_50_to_100, df_125_to_200, df_225_to_300]
targets = ['availability_30', 'availability_60', 'availability_90', 'availability_365']
names = ['listings', 'df_50_to_100', 'df_125_to_200', 'df_225_to_300']
for i in range(len(dfs)):
    for target in targets:
        X_train, X_test, y_train, y_test = train_test_split(dfs[i].drop(columns = cols), dfs[i][target], test_size=0.2, random_state=20166925)
        gb_regressor = GradientBoostingRegressor(n_estimators=400, max_depth=8, learning_rate=0.01, random_state=42)
        gb_regressor.fit(X_train, y_train)

            # Predict on the test set
        predictions = gb_regressor.predict(X_test)

            # Evaluate the model
            # mse = mean_squared_error(y_test, predictions)
            # print("Mean Squared Error:", mse)

            # print(y_test)
            # print(len(predictions))
            # print(len(df_test))
        mse = mean_squared_error(y_test, predictions)**0.5
        print('df: ', names[i])
        print('target: ', target)
        print('MSE: ', mse)